# Dir Structure
```
<your-repo>
|_ dataset
    |_ dataset.yaml
|_ yolov5*

```

In [1]:
# check if cuda is used
import torch
import os
from os import path
from PIL import Image
import matplotlib.pyplot as plt
import shutil

torch.cuda.is_available()

C:\Users\aditi\miniconda3\envs\yolo\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [5]:
print(torch.version.cuda)

11.3


In [ ]:
!python -m torch.utils.collect_env

In [6]:
# create dataset.yaml file
if not path.exists('datasets'):
    os.mkdir('datasets')

In [28]:
from PIL import Image
import matplotlib.pyplot as plt

# preprocess raw images
def load_image(path):
    size = (1280, 720)
    img = Image.open(path).convert('RGB')
    return img.resize(size)

# crop image for standard 1280x720
def crop_face(img):
    size = img.size
    center = img.size[0]*0.5
    w = size[1]*0.2
    
    left = center-w
    right = center+w
    top = 75
    bottom = 500
    
    img = img.crop((left,top,right,bottom))
    
    return img

def preprocess():
    DIR = './datasets/training/raw'
    SAVE = './datasets/training/images'
    for file in os.listdir(DIR):
        img = load_image(path.join(DIR, file))
        img = crop_face(img)
        img.save(f"{SAVE}/{file.split('.')[0]}_body.jpg")

In [2]:
# check image health (missing labels, missing images, etc.)
def check_labels():
    DIR = 'datasets/training_small'
    results = []
    for file in os.listdir(path.join(DIR, 'labels')):
        name = file.split('.')[0] + '.jpg'
        for root, dirs, files in os.walk(path.join(DIR, 'images')):
            if name not in files:
                results.append(name)
                
    if len(results) == 0:
        return 'labels are healthy'
    return results

def check_images():
    DIR = 'datasets/training_small'
    results = []
    for file in os.listdir(path.join(DIR, 'images')):
        name = file.split('.')[0] + '.txt'
        for root, dirs, files in os.walk(path.join(DIR, 'labels')):
            if name not in files:
                results.append(file)
    return results

def move_files(files):
    source = 'datasets/training_small/images'
    dest = 'datasets/training_small'
    for file in files:
        full_path = path.join(source, file)
        shutil.move(full_path, dest)

print('Images without label')
x = check_images()
print(len(x))
x

Images without label
7


['background_1.jpg',
 'background_2.jpg',
 'background_3.jpg',
 'background_4.jpg',
 'background_5.jpg',
 'background_6.jpg',
 'background_7.jpg']

# Template
`python yolov5/train.py --img 640 --batch 2 --epochs 1 --data datasets/training/ak.yaml --weights yolov5/yolov5s.pt --hyp hyp.custom.yaml`

In [ ]:
# training the model (remove mosaic)
python yolov5/train.py --img 640 --batch 2 --epochs 2 --data datasets/training/ak.yaml --weights yolov5/yolov5s.pt --hyp hyp.custom.yaml --rect

In [3]:
# config 
img_size = 640
batch_size = 2
epochs = 300
data_loc = 'datasets/training_small/ak.yaml'
weights = 'yolov5/yolov5s.pt'
hyperparam = 'hyp.custom.yaml'

In [ ]:
# training on smaller dataset (experimental)

# medium yolov5
python yolov5/train.py --img 640 --batch 2 --epochs 200 --data datasets/training_small/ak.yaml --weights yolov5/yolov5m.pt --hyp hyp.custom.yaml --rect

# small yolov5
python yolov5/train.py --img 640 --batch 2 --epochs 300 --data datasets/training_small/ak.yaml --weights yolov5/yolov5s.pt --hyp hyp.custom.yaml --rect

In [4]:
!python yolov5/train.py \
    --img {img_size} \
    --batch {batch_size} \
    --epochs {epochs} \
    --data {data_loc} \
    --weights {weights} \
    --hyp {hyperparam} \
    --rect

^C


In [35]:
# inference images need to have dimensions of 640x640 
def pre_inference():
    DIR = './datasets/inference/raw'
    SAVE = './datasets/inference/images'
    
    left = (0, 0, 640, 640)
    right = (640, 0, 1280, 640)
    crops = {'left': left, 'right': right}
    
    for file in os.listdir(DIR):
        img = load_image(path.join(DIR, file))
        
        for k, v in crops.items():
            proc = img.crop(v)
            proc.save(f"{SAVE}/{file.split('.')[0]}_{k}.jpg")
        
pre_inference()

In [1]:
import torch
model_path = 'yolov5/runs/train/training_test/weights/test.onnx'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True)



C:\Users\aditi\miniconda3\envs\yolo\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\aditi/.cache\torch\hub\master.zip
YOLOv5  2022-7-12 Python-3.9.12 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Loading yolov5\runs\train\training_test\weights\test.onnx for ONNX Runtime inference...
C:\Users\aditi\miniconda3\envs\yolo\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:55: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn("Specified provider '{}' is not in available provider names."
Adding AutoShape... 


In [3]:
model.conf = 0.02
img = 'datasets/inference/images/pull_3_right.jpg'
result = model(img)

result.print()

result.pandas().xyxy[0]

image 1/1: 640x640 1 hibiscus, 4 lavas
Speed: 9.0ms pre-process, 124.7ms inference, 3.0ms NMS per image at shape (1, 3, 640, 640)


,xmin,ymin,xmax,ymax,confidence,class,name
0,381.912292,199.964035,467.395813,295.345886,0.445480,1,lava
1,258.156494,202.558975,350.251648,310.185486,0.237979,0,hibiscus
2,270.278687,88.331223,589.577209,395.320007,0.027036,1,lava
3,403.665009,183.343689,495.277557,310.580383,0.022632,1,lava
4,359.010101,151.389618,459.982208,319.383850,0.021440,1,lava


In [5]:
result.xyxy[0]

tensor([[3.81912e+02, 1.99964e+02, 4.67396e+02, 2.95346e+02, 4.45480e-01, 1.00000e+00],
        [2.58156e+02, 2.02559e+02, 3.50252e+02, 3.10185e+02, 2.37979e-01, 0.00000e+00],
        [2.70279e+02, 8.83312e+01, 5.89577e+02, 3.95320e+02, 2.70360e-02, 1.00000e+00],
        [4.03665e+02, 1.83344e+02, 4.95278e+02, 3.10580e+02, 2.26324e-02, 1.00000e+00],
        [3.59010e+02, 1.51390e+02, 4.59982e+02, 3.19384e+02, 2.14396e-02, 1.00000e+00]], device='cuda:0')

In [16]:
model.forward

<bound method AutoShape.forward of AutoShape(
  (model): DetectMultiBackend()
)>

In [20]:
import numpy as np

arr = [[[1,2,3,4], [1,2,3,4], [1,2,3,4]]]
output = np.array(arr)
out = output.flatten()
output
out

array([1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4])

In [3]:
!python yolov5/detect.py \
            --weights yolov5/runs/train/exp2/weights/best.pt \
            --img 640 \
            --conf 0.75 \
            --source datasets/inference/images \
            --device cpu

detect: weights=['yolov5/runs/train/exp2/weights/best.pt'], source=datasets/inference/images, data=yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.75, iou_thres=0.45, max_det=1000, device=cpu, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.1-289-g526e650 Python-3.9.12 torch-1.11.0+cu113 CPU

Fusing layers... 
Model summary: 213 layers, 7053277 parameters, 0 gradients, 15.9 GFLOPs
image 1/29 D:\Repository\tracking-vision-yolo\datasets\inference\images\11629306773920_left.jpg: 640x640 Done. (0.129s)
image 2/29 D:\Repository\tracking-vision-yolo\datasets\inference\images\11629306773920_right.jpg: 640x640 1 midnight, 2 fangs, Done. (0.132s)
image 3/29 D:\Repository\tracking-vision-yolo\datasets\inference\images\1651165331840_left.j

In [4]:
!python yolov5/export.py --weights yolov5/runs/train/exp2/weights/best.pt --include onnx

export: data=D:\Repository\tracking-vision-yolo\yolov5\data\coco128.yaml, weights=['yolov5/runs/train/exp2/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, train=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5  v6.1-289-g526e650 Python-3.9.12 torch-1.11.0+cu113 CPU

Fusing layers... 
Model summary: 213 layers, 7053277 parameters, 0 gradients, 15.9 GFLOPs

PyTorch: starting from yolov5\runs\train\exp2\weights\best.pt with output shape (1, 25200, 21) (13.8 MB)

ONNX: starting export with onnx 1.11.0...
ONNX: export success, saved as yolov5\runs\train\exp2\weights\best.onnx (27.3 MB)

Export complete (2.10s)
Results saved to D:\Repository\tracking-vision-yolo\yolov5\runs\train\exp2\weights
Detect:          python detect.py --weights yolov5\runs\train\exp2\weigh

In [14]:
import json
import os
from os import path

# converts classes.txt to JSON
def create_dict():
    FILE_PATH = "./datasets/training_small/labels/classes.txt"
    f = open(FILE_PATH, "r")
    classes_dict = {}
    for i, name in enumerate(f.readlines()):
        class_obj = {"name": name.strip()}
        classes_dict[i] = class_obj
        
    return classes_dict

x = json.dumps(create_dict())
f = open("classes.json", "w")
f.write(x)
print(x)

{"0": {"name": "midnight"}, "1": {"name": "lava"}, "2": {"name": "spot"}, "3": {"name": "cardigan"}, "4": {"name": "beagle"}, "5": {"name": "steward"}, "6": {"name": "plume"}, "7": {"name": "hibiscus"}, "8": {"name": "orchid"}, "9": {"name": "popukar"}, "10": {"name": "ansel"}, "11": {"name": "vanilla"}, "12": {"name": "fang"}, "13": {"name": "kroos"}, "14": {"name": "melantha"}, "15": {"name": "catapult"}}
